In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

In [4]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [5]:
# ratings
cnt = 0
for i in ratings:
    if cnt != 10:
        print(i)
        cnt +=1
    else:
        break

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Strictly Ballroom (1992)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'92'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Very Brady Sequel, A (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'301'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Pulp Fiction (1994)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'60'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Scream 2 (1997)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'197'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Crash (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'601'>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Aladdin (1992)'>, 'user_id': <tf.Tensor: shape=(), 

In [6]:
# movies
cnt = 0
for i in movies:
    if cnt != 10:
        print(i)
        cnt +=1
    else:
        break

tf.Tensor(b'You So Crazy (1994)', shape=(), dtype=string)
tf.Tensor(b'Love Is All There Is (1996)', shape=(), dtype=string)
tf.Tensor(b'Fly Away Home (1996)', shape=(), dtype=string)
tf.Tensor(b'In the Line of Duty 2 (1987)', shape=(), dtype=string)
tf.Tensor(b'Niagara, Niagara (1997)', shape=(), dtype=string)
tf.Tensor(b"Young Poisoner's Handbook, The (1995)", shape=(), dtype=string)
tf.Tensor(b'Age of Innocence, The (1993)', shape=(), dtype=string)
tf.Tensor(b'Flirt (1995)', shape=(), dtype=string)
tf.Tensor(b'Frisk (1995)', shape=(), dtype=string)
tf.Tensor(b'unknown', shape=(), dtype=string)


In [7]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
print(user_ids_vocabulary.get_vocabulary()[0:10])

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)
print(movie_titles_vocabulary.get_vocabulary()[0:10])

['[UNK]', '405', '655', '13', '450', '276', '416', '537', '303', '234']
['[UNK]', "Ulee's Gold (1997)", 'That Darn Cat! (1997)', 'Substance of Fire, The (1996)', 'Sliding Doors (1998)', 'Nightwatch (1997)', 'Money Talks (1997)', 'Kull the Conqueror (1997)', 'Ice Storm, The (1997)', 'Hurricane Streets (1998)']


In [8]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [9]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [10]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3


25/25 [==============================] - 14s 553ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0046 - factorized_top_k/top_50_categorical_accuracy: 0.0433 - factorized_top_k/top_100_categorical_accuracy: 0.0981 - loss: 33076.6864 - regularization_loss: 0.0000e+00 - total_loss: 33076.6864
Epoch 2/3
25/25 [==============================] - 14s 556ms/step - factorized_top_k/top_1_categorical_accuracy: 1.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0142 - factorized_top_k/top_50_categorical_accuracy: 0.1047 - factorized_top_k/top_100_categorical_accuracy: 0.2109 - loss: 31011.7233 - regularization_loss: 0.0000e+00 - total_loss: 31011.7233
Epoch 3/3
25/25 [==============================] - 14s 558ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorical_accur